In [17]:
!pip3 install pandas
!pip3 install numpy
!pip3 install -U sentence-transformers
!pip3 install umap-learn

In [18]:
# Настройка устройства
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используется: {device}")

Используется: cuda


In [19]:
import os
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

#заранее забиваю настройки
model_name = 'paraphrase-multilingual-mpnet-base-v2'
batch_size = 512  
output_dir = 'embeddings_batches'  # Папка создастся там же, где ноутбук
os.makedirs(output_dir, exist_ok=True)

#загружаю модель
model = SentenceTransformer(model_name, device=device)

In [ ]:
# считываем данные
df = pd.read_csv('clean_train_final.csv')

#смотрим количество строк с пропущенными данными
print(df.isnull().sum())

name                     0
type                     0
category_l2              0
category_l4              0
is_markup                0
target                   0
stratify_column          0
name_processed           0
type_processed           0
category_l2_processed    0
category_l4_processed    0
dtype: int64


In [21]:
#функция для обработки батчей 
def process_dataset(df, text_column='name_processed', type_column='type_processed'):
    # Создаем подпапки для разных типов эмбеддингов
    name_output_dir = os.path.join(output_dir, 'name_embeddings')
    type_output_dir = os.path.join(output_dir, 'type_embeddings')
    os.makedirs(name_output_dir, exist_ok=True)
    os.makedirs(type_output_dir, exist_ok=True)

    for i in tqdm(range(0, len(df), batch_size), desc='Обработка батчей'):
        batch = df.iloc[i:i+batch_size]
        
        # Эмбеддинги для name
        name_texts = batch[text_column].tolist()
        name_embeddings = model.encode(name_texts, normalize_embeddings=False, show_progress_bar=False)
        np.save(f'{name_output_dir}/name_embeddings_batch_{i//batch_size}.npy', name_embeddings)
        
        # Эмбеддинги для type
        type_texts = batch[type_column].tolist()
        type_embeddings = model.encode(type_texts, normalize_embeddings=False, show_progress_bar=False)
        np.save(f'{type_output_dir}/type_embeddings_batch_{i//batch_size}.npy', type_embeddings)

process_dataset(df)
print(f"✅ Эмбеддинги сохранены в:")
print(f"- Name: {os.path.abspath(os.path.join(output_dir, 'name_embeddings'))}")
print(f"- Type: {os.path.abspath(os.path.join(output_dir, 'type_embeddings'))}")

Обработка батчей: 100%|██████████| 1398/1398 [05:41<00:00,  4.09it/s]

✅ Эмбеддинги сохранены в:
- Name: c:\Users\mrskb\Ucheba\hackathon\embeddings_batches\name_embeddings
- Type: c:\Users\mrskb\Ucheba\hackathon\embeddings_batches\type_embeddings


In [22]:
# Объединяем эмбеддинги из батчей в один файл

def merge_embeddings(output_dir, prefix='embeddings_batch_'):
    
    # Проверяем существование папки
    if not os.path.exists(output_dir):
        raise FileNotFoundError(f"Папка {output_dir} не существует")

    # Получаем и фильтруем файлы
    all_files = [f for f in os.listdir(output_dir) if f.startswith(prefix)]
    
    # Сортировка
    def sort_key(x):
        try:
            # Пробуем извлечь число из имени файла
            return int(''.join(filter(str.isdigit, x)))
        except:
            return 0
            
    all_files = sorted(all_files, key=sort_key)
    
    if not all_files:
        raise ValueError(f"Нет файлов с префиксом {prefix} в {output_dir}")

    # Загрузка и объединение
    final_embeddings = []
    for file in tqdm(all_files, desc=f'Объединение {prefix}'):
        batch = np.load(os.path.join(output_dir, file))
        final_embeddings.append(batch)
    
    final_embeddings = np.concatenate(final_embeddings)
    
    # Сохранение
    output_path = f'final_{prefix.split("_")[0]}.npy'  # final_name.npy или final_type.npy
    np.save(output_path, final_embeddings)
    print(f"✅ Файл '{output_path}' сохранён: {os.path.abspath(output_path)}")
    return final_embeddings

In [23]:
# Для эмбеддингов name
name_embeddings = merge_embeddings(
    output_dir=r'C:\Users\mrskb\Ucheba\hackathon\embeddings_batches\name_embeddings',
    prefix='name_embeddings_batch_'  # Точное совпадение с началом имен файлов
)

# Для эмбеддингов type
type_embeddings = merge_embeddings(
    output_dir=r'C:\Users\mrskb\Ucheba\hackathon\embeddings_batches\type_embeddings',
    prefix='type_embeddings_batch_'
)

Объединение name_embeddings_batch_: 100%|██████████| 1398/1398 [00:05<00:00, 269.50it/s]


✅ Файл 'final_name.npy' сохранён: c:\Users\mrskb\Ucheba\hackathon\final_name.npy


Объединение type_embeddings_batch_: 100%|██████████| 1398/1398 [00:05<00:00, 268.79it/s]


✅ Файл 'final_type.npy' сохранён: c:\Users\mrskb\Ucheba\hackathon\final_type.npy
